In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix, make_scorer
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier, GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
import xgboost as xgb
from pyearth import Earth

In [3]:
nba_2017_2018 = pd.read_csv('archive-2/NBA Players Stats 201718.csv')
nba_2017_2018 = nba_2017_2018.drop(columns=['Rk','Tm', 'Pos'], axis=1)
nba_2017_2018['Player'] = nba_2017_2018['Player'].str.split('\\').str[0]
nba_2017_2018.fillna(0, inplace=True)
nba_2017_2018 = nba_2017_2018[nba_2017_2018['G'] >= 20]
cols_to_keep = ['Player', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
                '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
                'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

# Group the data by player and calculate the weighted average and sum of the other columns
nba_2017_2018 = nba_2017_2018.groupby(['Player', 'Age'])[cols_to_keep].agg({'G': 'sum', 'GS': 'sum', 
                                                                                     'MP': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'FG': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'FGA': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'FG%': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     '3P': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     '3PA': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     '3P%': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     '2P': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     '2PA': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     '2P%': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'eFG%': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'FT': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'FTA': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'FT%': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'ORB': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'DRB': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'TRB': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'AST': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'STL': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']),
                                                                                     'BLK': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']),
                                                                                     'TOV': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']),
                                                                                     'PF': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']),
                                                                                     'PTS': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G'])}).reset_index()

percentage_columns = ['FG%', '3P%', '2P%', 'eFG%', 'FT%']
for col in percentage_columns:
    nba_2017_2018[col] = nba_2017_2018[col] * 100

nba_2017_2018 = nba_2017_2018.round(2)

all_star_list = ['Kemba Walker', 'Kyrie Irving', 'Kawhi Leonard', 'Giannis Antetokounmpo', 
                'Joel Embiid', 'Kyle Lowry', 'Victor Oladipo', 'Khris Middleton', 'Bradley Beal',
                'Ben Simmons', 'Blake Griffin', 'Nikola Vučević', "D'Angelo Russell",
                'Stephen Curry', 'James Harden', 'Kevin Durant', 'Paul George', 'LeBron James',
                'Russell Westbrook', 'Damian Lillard', 'Klay Thompson', 'Anthony Davis', 
                'LaMarcus Aldridge', 'DeMarcus Cousins', 'Nikola Jokić', 'Karl-Anthony Towns']
nba_2017_2018['AS_NY'] = nba_2017_2018['Player'].apply(lambda x: 1 if x in all_star_list else 0)

In [ ]:
nba_2017_2018

In [4]:
nba_2018_2019 = pd.read_csv('archive-2/NBA Players Stats 201819.csv')
nba_2018_2019 = nba_2018_2019.drop(columns=['Rk','Tm','Pos'], axis=1)
nba_2018_2019['Player'] = nba_2018_2019['Player'].str.split('\\').str[0]
nba_2018_2019.fillna(0, inplace=True)
nba_2018_2019 = nba_2018_2019[nba_2018_2019['G'] >= 20]
cols_to_keep = ['Player', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
                '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
                'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

# Group the data by player and calculate the weighted average and sum of the other columns
nba_2018_2019 = nba_2018_2019.groupby(['Player', 'Age'])[cols_to_keep].agg({'G': 'sum', 'GS': 'sum', 
                                                                                     'MP': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'FG': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'FGA': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'FG%': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     '3P': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     '3PA': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     '3P%': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     '2P': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     '2PA': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     '2P%': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'eFG%': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'FT': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'FTA': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'FT%': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'ORB': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'DRB': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'TRB': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'AST': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'STL': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']),
                                                                                     'BLK': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']),
                                                                                     'TOV': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']),
                                                                                     'PF': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']),
                                                                                     'PTS': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G'])}).reset_index()

percentage_columns = ['FG%', '3P%', '2P%', 'eFG%', 'FT%']
for col in percentage_columns:
    nba_2018_2019[col] = nba_2018_2019[col] * 100


nba_2018_2019 = nba_2018_2019.round(2)

all_star_list = ['Kemba Walker', 'Trae Young', 'Giannis Antetokounmpo', 'Pascal Siakam', 'Joel Embiid',
                'Kyle Lowry', 'Ben Simmons', 'Jimmy Butler', 'Khris Middleton', 'Bam Adebayo', 
                'Jayson Tatum', 'Domantas Sabonis', 'James Harden', 'Luka Dončić', 'LeBron James',
                'Kawhi Leonard', 'Anthony Davis', 'Chris Paul', 'Russell Westbrook', 'Damian Lillard', 
                'Donovan Mitchell', 'Brandon Ingram', 'Nikola Jokić', 'Rudy Gobert', 'Devin Booker']
nba_2018_2019['AS_NY'] = nba_2018_2019['Player'].apply(lambda x: 1 if x in all_star_list else 0)

In [ ]:
nba_2018_2019

**Naive Model**

In [ ]:
X_train = nba_2017_2018.drop(['Player', 'AS_NY'], axis=1) 
y_train = nba_2017_2018['AS_NY']  

X_test = nba_2018_2019.drop(['Player', 'AS_NY'], axis=1)  
y_test = nba_2018_2019['AS_NY']  

y_pred = [0] * len(y_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("Confusion Matrix:")
print(confusion_mat)

**Base Logistic Regression Model**

In [ ]:
logreg = LogisticRegression(max_iter=10000)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("Confusion Matrix:")
print(confusion_mat)

**Feature Importances**

In [ ]:
coefficients = logreg.coef_[0]


feature_names = X_train.columns


feature_importance = sorted(zip(feature_names, coefficients), key=lambda x: abs(x[1]), reverse=True)


df_feature_importance = pd.DataFrame(feature_importance, columns=['Feature', 'Coefficient'])


df_feature_importance['Absolute Importance'] = df_feature_importance['Coefficient'].abs()
df_feature_importance = df_feature_importance.sort_values(by='Absolute Importance', ascending=False)
df_feature_importance